## Loading Data

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras import layers
from tensorflow import data as tf_data
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

2023-12-10 00:36:08.403629: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-10 00:36:08.446134: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 00:36:08.446165: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 00:36:08.446190: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 00:36:08.454402: I tensorflow/core/platform/cpu_feature_g

In [2]:
data_augmentation=keras.Sequential([
    layers.RandomContrast(0.2),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2)
])

2023-12-10 00:37:22.417803: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14774 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-12-10 00:37:22.418370: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 4130 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:73:00.0, compute capability: 7.5


In [3]:
# tfds.disable_progress_bar()
train_60,val_20,test_20=tfds.load("oxford_iiit_pet",split=["train+test[:20%]","test[20%:60%]","test[60%:]"],as_supervised=True)

resizing=layers.Resizing(224,224)
train_60=train_60.map(lambda x,y:(resizing(x),y),num_parallel_calls=tf_data.AUTOTUNE)
val_20=val_20.map(lambda x,y:(resizing(x),y),num_parallel_calls=tf_data.AUTOTUNE)
test_20=test_20.map(lambda x,y:(resizing(x),y),num_parallel_calls=tf_data.AUTOTUNE)

train_60_1=train_60.map(lambda x,y:(data_augmentation(x,training=True),y),num_parallel_calls=tf_data.AUTOTUNE)

train_60=train_60.concatenate(train_60_1).shuffle(1000)

batch_size = 128

train_60 = train_60.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
val_20 = val_20.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
test_20 = test_20.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()

In [2]:
train,test=tfds.load("oxford_iiit_pet",split=["train","test"],as_supervised=True)

resizing=layers.Resizing(224,224)
train=train.map(lambda x,y:(resizing(x),y))
test=test.map(lambda x,y:(resizing(x),y))

batch_size = 64

train = train.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
test = test.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()

In [5]:
F10_train=tfds.load("oxford_iiit_pet",split=[f"train[:{r}%]+train[{r+10}%:]" for r in range(0,100,10)],as_supervised=True)
F10_val=tfds.load("oxford_iiit_pet",split=[f"train[{r}%:{r+10}%]" for r in range(0,100,10)],as_supervised=True)

## Model

In [4]:
base_model=keras.applications.ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3),pooling="avg")

In [5]:
base_model.trainable=False

In [6]:
inputs=keras.Input(shape=(224,224,3))   
preprocessing=tf.keras.applications.resnet.preprocess_input(inputs)
x=base_model(preprocessing, training=False)
x=layers.Dense(256,activation="relu")(x)
x=layers.Dropout(0.5)(x)    
outputs=layers.Dense(37)(x)

model=keras.Model(inputs,outputs)

loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizers=keras.optimizers.Adam(learning_rate=0.0002,weight_decay=0.0005)

model.compile(optimizer=optimizers,loss=loss_fn,metrics=[keras.metrics.SparseTopKCategoricalAccuracy( k=1, name="accuracy")])

In [7]:
model.summary(show_trainable=True) 

Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         Y          
                                                                            
 tf.__operators__.getitem (  (None, 224, 224, 3)       0         Y          
 SlicingOpLambda)                                                           
                                                                            
 tf.nn.bias_add (TFOpLambda  (None, 224, 224, 3)       0         Y          
 )                                                                          
                                                                            
 resnet50 (Functional)       (None, 2048)              2358771   N          
                                                       2                    
                                                             

In [8]:
callback=EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)
history=model.fit(train_60,epochs=50,validation_data=val_20)

Epoch 1/50


Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
2023-12-10 00:17:10.815856: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-12-10 00:17:14.471263: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fdb8800cbb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-10 00:17:14.471336: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2023-12-10 00:17:14.471360: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Quadro RTX 5000, Compute Capability 7.5
2023-12-10 00:17:14.489563: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-10 00:17:14.635273: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using 

31/69 [============>.................] - ETA: 9s - loss: 3.2754 - accuracy: 0.1578

Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment


69/69 [==============================] - 34s 370ms/step - loss: 2.6385 - accuracy: 0.3100 - val_loss: 1.0983 - val_accuracy: 0.7873
Epoch 2/50
69/69 [==============================] - 21s 298ms/step - loss: 1.1565 - accuracy: 0.6675 - val_loss: 0.6052 - val_accuracy: 0.8521
Epoch 3/50
69/69 [==============================] - 19s 273ms/step - loss: 0.7532 - accuracy: 0.7797 - val_loss: 0.4776 - val_accuracy: 0.8698
Epoch 4/50
69/69 [==============================] - 18s 267ms/step - loss: 0.6003 - accuracy: 0.8198 - val_loss: 0.4137 - val_accuracy: 0.8834
Epoch 5/50
69/69 [==============================] - 18s 264ms/step - loss: 0.4851 - accuracy: 0.8527 - val_loss: 0.3951 - val_accuracy: 0.8793
Epoch 6/50
69/69 [==============================] - 18s 263ms/step - loss: 0.4142 - accuracy: 0.8742 - val_loss: 0.3751 - val_accuracy: 0.8862
Epoch 7/50
69/69 [==============================] - 18s 268ms/step - loss: 0.3585 - accuracy: 0.8896 - val_loss: 0.3679 - val_accuracy: 0.8903
Epoch 8/50

In [11]:
model.evaluate(test_20)   

12/12 [==============================] - 2s 192ms/step - loss: 0.2746 - accuracy: 0.9149


[0.2746414244174957, 0.914850115776062]

In [14]:
model.save("model.h5")

/home/nima/miniconda3/envs/BrainNeoCare/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
model=keras.models.load_model("model.h5")

In [5]:
model.evaluate(test_20)

2023-12-10 00:40:14.208189: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


12/12 [==============================] - 9s 333ms/step - loss: 0.2746 - accuracy: 0.9149


[0.2746414244174957, 0.914850115776062]

In [6]:
model.summary(show_trainable=True)

Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         Y          
                                                                            
 tf.__operators__.getitem (  (None, 224, 224, 3)       0         Y          
 SlicingOpLambda)                                                           
                                                                            
 tf.nn.bias_add (TFOpLambda  (None, 224, 224, 3)       0         Y          
 )                                                                          
                                                                            
 resnet50 (Functional)       (None, 2048)              2358771   Y          
                                                       2                    
                                                             

In [7]:
model.trainable=True
model.summary(show_trainable=True)

Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         Y          
                                                                            
 tf.__operators__.getitem (  (None, 224, 224, 3)       0         Y          
 SlicingOpLambda)                                                           
                                                                            
 tf.nn.bias_add (TFOpLambda  (None, 224, 224, 3)       0         Y          
 )                                                                          
                                                                            
 resnet50 (Functional)       (None, 2048)              2358771   Y          
                                                       2                    
                                                             

In [8]:
loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizers=keras.optimizers.Adam(learning_rate=0.000001)

model.compile(optimizer=optimizers,loss=loss_fn,metrics=[keras.metrics.SparseTopKCategoricalAccuracy( k=1, name="accuracy")])

In [9]:
callback=EarlyStopping(monitor="val_accuracy",patience=10)
model.fit(train_60,epochs=50,validation_data=val_20)

Epoch 1/50


Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
2023-12-10 00:42:21.173842: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa23431d7e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-10 00:42:21.173882: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2023-12-10 00:42:21.173887: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Quadro RTX 5000, Compute Capability 7.5
2023-12-10 00:42:21.179583: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-10 00:42:21.327460: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-12-10 00:42:25.826891: W tensorflow/tsl/f

29/69 [===========>..................] - ETA: 29s - loss: 0.0210 - accuracy: 0.9954

Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment


68/69 [============================>.] - ETA: 0s - loss: 0.0963 - accuracy: 0.9720

2023-12-10 00:43:33.371913: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.77GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-10 00:43:33.371964: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.77GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


69/69 [==============================] - 94s 882ms/step - loss: 0.0976 - accuracy: 0.9711 - val_loss: 0.3655 - val_accuracy: 0.8991
Epoch 2/50
69/69 [==============================] - 55s 798ms/step - loss: 0.0747 - accuracy: 0.9771 - val_loss: 0.3714 - val_accuracy: 0.9012
Epoch 3/50
69/69 [==============================] - 55s 794ms/step - loss: 0.0648 - accuracy: 0.9802 - val_loss: 0.3752 - val_accuracy: 0.8991
Epoch 4/50
69/69 [==============================] - 54s 788ms/step - loss: 0.0628 - accuracy: 0.9806 - val_loss: 0.3779 - val_accuracy: 0.9005
Epoch 5/50
69/69 [==============================] - 55s 790ms/step - loss: 0.0571 - accuracy: 0.9818 - val_loss: 0.3782 - val_accuracy: 0.9025
Epoch 6/50
69/69 [==============================] - 54s 790ms/step - loss: 0.0460 - accuracy: 0.9857 - val_loss: 0.3870 - val_accuracy: 0.8978
Epoch 7/50
69/69 [==============================] - 54s 782ms/step - loss: 0.0467 - accuracy: 0.9849 - val_loss: 0.3834 - val_accuracy: 0.8978
Epoch 8/50

In [10]:
model.evaluate(test_20)

12/12 [==============================] - 2s 193ms/step - loss: 0.3357 - accuracy: 0.9217


[0.33572977781295776, 0.9216621518135071]

In [11]:
model.save("fine_tuned_model.h5")

/home/nima/miniconda3/envs/BrainNeoCare/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
#to reset the gpu memory
from numba import cuda 
device = cuda.get_current_device()
device.reset()